###### imports

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from matplotlib import transforms
import scipy
import scipy.stats as st
from SequenceGenerator import MultiSequenceGenerator
from scipy.integrate import odeint
import networkx as nx
import random
import matplotlib.colors as mcolors
import matplotlib.cm as cm

# generating bianconi-barabasi

### implementation

In [ ]:
class BianconiBarabashi():
    def draw_graph_for_time_slot(self, step_number):
        print(f'step number: {step_number}')
        fig = plt.figure(figsize=(50, 50))
        degrees = np.array([self.graph.degree(n) for n in self.graph.nodes()])
        node_size = degrees*100
        
        pos=nx.spring_layout(self.graph)
        cmap=plt.cm.viridis
        
        nodes = nx.draw_networkx_nodes(self.graph, pos, node_size=node_size, node_color=node_size, cmap=cmap)
        edges = nx.draw_networkx_edges(self.graph, pos)
        
        plt.colorbar(nodes)
        plt.axis('off')
        plt.title(f'step number: {step_number}')
        plt.show()

    
    
    def append_nodes(self):
        time_intervals = int(self.step_number/self.time_slot_number)
        time_slots = list(range(time_intervals, self.step_number+1, time_intervals))
        
        for new_node in range(self.initial_node_number+1, self.step_number+self.initial_node_number+1):
            degrees = [val for (node, val) in sorted(self.graph.degree(), key=lambda pair: pair[0])]
            summation = np.sum([self.fitness[i]*degrees[i] for i in range(len(degrees))])
            probabilities = [(self.fitness[i]*degrees[i])/summation for i in range(len(degrees))]
            probabilities_cum = np.cumsum(probabilities)
            interval_max = probabilities_cum[len(probabilities_cum) - 1]
            
            self.graph.add_node(new_node)
            
            for _ in range(self.new_node_link):
                random_number = np.random.uniform(0.0, interval_max)
                connected_node = sum(i < random_number for i in probabilities_cum) + 1 # pluse one is because node numbers starts from 1 not zero
                self.graph.add_edge(new_node, connected_node)

            step_number = new_node - (self.initial_node_number+1)
            if (step_number in time_slots):
                self.draw_graph_for_time_slot(step_number)
    
    def __init__(self, initial_node_number=3, step_number=100, new_node_link=3, fitness_function=np.random.gamma, fitness_function_parameters=[], time_slot_number=10):
        self.initial_node_number=initial_node_number
        self.step_number=step_number
        self.new_node_link=new_node_link
        self.fitness_function_parameters = fitness_function_parameters.copy()
        self.fitness_function_parameters.append(self.initial_node_number+self.step_number)
        self.fitness_function = fitness_function
        self.fitness = self.fitness_function(*self.fitness_function_parameters)
        self.time_slot_number = time_slot_number
        self.graph = nx.complete_graph(self.initial_node_number+1)
        self.graph.remove_node(0)
        self.append_nodes()
        self.set_degree_distribution()

    def set_degree_distribution(self):
        self.degree_distribution = np.array([self.graph.degree(n) for n in self.graph.nodes()])

    def draw_degree_distribution(self):
        sns.displot(self.degree_distribution, kde=True)
        plt.title('degree distribution')
        plt.xlabel('degree')
        plt.ylabel('occurance of each degree')
        plt.show()

        sns.displot(self.degree_distribution, kde=True)
        plt.title('degree distribution semilog')
        plt.xlabel('degree')
        plt.ylabel('occurance of each degree')
        plt.semilogy()
        plt.show()

        sns.displot(self.degree_distribution, kde=True)
        plt.title('degree distribution log-log')
        plt.xlabel('degree')
        plt.ylabel('occurance of each degree')
        plt.xscale('log')
        plt.yscale('log')
        plt.show()

    def draw_graph(self):
        fig = plt.figure(figsize=(50, 50))
        node_size = self.degree_distribution*100
        
        pos=nx.spring_layout(self.graph)
        cmap=plt.cm.viridis
        
        nodes = nx.draw_networkx_nodes(self.graph, pos, node_size=node_size, node_color=node_size, cmap=cmap)
        edges = nx.draw_networkx_edges(self.graph, pos)
        
        plt.colorbar(nodes)
        plt.axis('off')
        plt.show()

### simulation

#### gamma as fitness function

###### parameters

In [ ]:
initi_node_number = 3
step_number = 1000
new_node_link = 3

# gamma as fitness function
shape, scale = 2., 2.
fitness_function = np.random.gamma
fitness_function_parameters = [shape, scale]

###### generating graph

In [ ]:
g_holder = BianconiBarabashi(initi_node_number, step_number, new_node_link, fitness_function=fitness_function, fitness_function_parameters=fitness_function_parameters)

###### degree distribution

In [ ]:
g_holder.draw_degree_distribution()

###### graph

In [ ]:
g_holder.draw_graph()

#### log-normal as fitness function

###### parameters

In [ ]:
initi_node_number = 3
step_number = 1000
new_node_link = 3

# log-normal as fitness
mu, sigma = 3., 1. # mean and standard deviation
fitness_function = np.random.lognormal
fitness_function_parameters = [mu, sigma]

###### generating graph

In [ ]:
g_holder = BianconiBarabashi(initi_node_number, step_number, new_node_link, fitness_function=fitness_function, fitness_function_parameters=fitness_function_parameters)

###### degree distribution

In [ ]:
g_holder.draw_degree_distribution()

###### graph

In [ ]:
g_holder.draw_graph()

#### power-law as fitness

###### parameters

In [ ]:
initi_node_number = 3
step_number = 1000
new_node_link = 3

# power-law as fitness function
a = 5. # shape
fitness_function = np.random.power
fitness_function_parameters = [a]

###### generating graph

In [ ]:
g_holder = BianconiBarabashi(initi_node_number, step_number, new_node_link, fitness_function=fitness_function, fitness_function_parameters=fitness_function_parameters)

###### degree distribution

In [ ]:
g_holder.draw_degree_distribution()

###### graph

In [ ]:
g_holder.draw_graph()